In [1]:
%load_ext autoreload
%autoreload 2

## Standard libraries
import os
from collections import defaultdict

# 3rd party libraries
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image

## PyTorch
import torch.nn.functional as F
import torch

## our own modules
from utils import get_percentage_of_image, normalize_image, generate_adversarial_examples, create_image_dict, read_conf_from_dotenv, get_dir_path

conf = read_conf_from_dotenv()


In [11]:
# Constants for Model and XAI Configuration
MODEL_TYPE = "simple_cnn"  # Model architecture being used
XAI_TYPE = "gradcam"  # XAI method being applied

# Base directory for dataset and output paths
BASE_DIR = conf.BASE_DIR
DATASET = "atsds_large"
DATASET_SPLIT = "test"

# Paths for dataset and associated outputs
GROUND_TRUTH_DIR = get_dir_path(BASE_DIR, f"{DATASET}_mask", DATASET_SPLIT)
BACKGROUND_DIR = get_dir_path(BASE_DIR, f"{DATASET}_background", DATASET_SPLIT)
DATASET_DIR = get_dir_path(BASE_DIR, DATASET, DATASET_SPLIT)
# XAI_DIR = get_dir_path(BASE_DIR, "auswertung_hpc", "auswertung", MODEL_TYPE, XAI_TYPE, DATASET_SPLIT, check_exists=False)
XAI_DIR = get_dir_path(BASE_DIR, "XAI_results", MODEL_TYPE, XAI_TYPE, DATASET_SPLIT)
ADV_FOLDER = get_dir_path(BASE_DIR, "XAI_evaluation", MODEL_TYPE, XAI_TYPE, "revelation", check_exists=False)


In [12]:
# Create folder structure for Revelation Dataset
CATEGORIES, imagedict = create_image_dict(BASE_DIR, DATASET, DATASET_SPLIT)
print(CATEGORIES)

['00001', '00002', '00003', '00004', '00005', '00007', '00008', '00009', '00010', '00011', '00012', '00013', '00014', '00017', '00018', '00025', '00031', '00035', '00038']


In [6]:
DATASET_DIR

'/home/ck/mnt/XAI-DIA-gl/Julian/Dataset_Masterarbeit/atsds_large/test'

In [ ]:
# The generation of the evaluation-images takes about 36 min.

In [13]:
revelation_condition = lambda adv_mask: adv_mask == 1

generate_adversarial_examples(
    adv_folder=ADV_FOLDER,
    pct_range=range(0, 101, 10),
    categories=CATEGORIES,
    imagedict=imagedict,
    #img_path=IMAGES_PATH,
    img_path=DATASET_DIR,
    background_dir=BACKGROUND_DIR,
    xai_dir=XAI_DIR,
    # this
    mask_condition=revelation_condition
)

Processing percentage: 0%


/home/ck/iee-ge/repos/MT_JU_XAI_traffic_signs/Code/utils.py:60: RuntimeWarning: invalid value encountered in divide
  return np.nan_to_num((img-img.min())/(img.max()-img.min()), nan=0.0, posinf=0.0,neginf=0.0)


Processing percentage: 10%
Processing percentage: 20%
Processing percentage: 30%
Processing percentage: 40%
Processing percentage: 50%
Processing percentage: 60%
Processing percentage: 70%
Processing percentage: 80%
Processing percentage: 90%
Processing percentage: 100%


In [18]:
pjoin = os.path.join
im1_raw = np.load(pjoin(XAI_DIR, "00001/mask/000035.png.npy"))

im2_raw = Image.open(pjoin(GROUND_TRUTH_DIR, "00001/000035.png"))
im3_raw = Image.open(pjoin(BACKGROUND_DIR, "00001/000035.png"))
im4_raw = Image.open(pjoin(DATASET_DIR,  "00001/000035.png"))
im1 = np.array(np.array(im1_raw))
im4 = np.array(im4_raw)
im3 = np.array(im3_raw)
im2 = np.array(im2_raw)/255
im2_gs = np.array(im2_raw.convert('L'))

In [19]:
fig, axes = plt.subplots(2, 2, figsize=(8, 8))

axes[0, 0].imshow(im1)
axes[0, 0].axis('off')  # remove axis

axes[0, 1].imshow(im2)
axes[0, 1].axis('off')  # remove axis

axes[1, 0].imshow(im3)
axes[1, 0].axis('off')  # remove axis

axes[1, 1].imshow(im4)
axes[1, 1].axis('off')  # remove axis

plt.show()
